<a href="https://colab.research.google.com/github/bekka9/Optimisation_Methods_gradient/blob/main/MO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math as m
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from numpy import linalg

import scipy as sp
from scipy import optimize

a = 0
b = np.pi
eps = 1/(16**4/2)
n = 3
#h = (b - a)/n
h = 1/(16**3*2)
#eps = 1/(16**3)
k_iter = 0
complexity = 0

def f_(y, i_num, x, complexity):
    complexity += 1
    for i in range (n):
      if (i == i_num):
        x[i] = y
    return (x[0] - 2)**2+ 2 + x[1]*3, complexity
def f(y, i_num, x, complexity):
    complexity += 1
    for i in range (n):
      if (i == i_num):
        x[i] = y
    return np.exp(0.6*x[0]) + np.exp(-0.6*x[0]) + np.exp(0.1*x[1]) + np.exp(-0.1*x[1]) + np.exp(0.11*x[2]) + np.exp(-0.11*x[2]) + 0.6*x[0] + 0.1*x[1] + 0.11*x[2] , complexity

def f_1(x):
    return np.exp(0.6*x[0]) + np.exp(-0.6*x[0]) + np.exp(0.1*x[1]) + np.exp(-0.1*x[1]) + np.exp(0.11*x[2]) + np.exp(-0.11*x[2]) + 0.6*x[0] + 0.1*x[1] + 0.11*x[2]

def df_x(x, complexity):
    complexity += 1
    return (0.6 * np.exp(0.6*x[0]) - 0.6 * np.exp(-0.6*x[0]) + 0.6) , complexity

def df_y(x, complexity):
    complexity += 1
    return (0.1 * np.exp(0.1*x[1]) - 0.1 * np.exp(-0.1*x[1]) + 0.1) , complexity

def df_z(x, complexity):
    complexity += 1
    return (0.11 * np.exp(0.11*x[2]) - 0.11 * np.exp(-0.11*x[2]) + 0.11) , complexity


def df_(df, x, complexity):
    complexity += 1
    df[0] = 0.6 * np.exp(0.6*x[0]) - 0.6 * np.exp(-0.6*x[0]) + 0.6
    df[1] = 0.1 * np.exp(0.1*x[1]) - 0.1 * np.exp(-0.1*x[1]) + 0.1
    df[2] = 0.11 * np.exp(0.11*x[2]) - 0.11 * np.exp(-0.11*x[2]) + 0.11
    return df, complexity

def df_2(df, x):
    df[0] = 0.6**2 * np.exp(0.6*x[0]) + 0.6**2 * np.exp(-0.6*x[0])
    df[1] = 0.1**2 * np.exp(0.1*x[1]) + 0.1**2 * np.exp(-0.1*x[1])
    df[2] = 0.11**2 * np.exp(0.11*x[2]) + 0.11**2 * np.exp(-0.11*x[2])
    return df


def round_hex(x, to=5):
    x = [float(x[0]).hex(), float(x[1]).hex(), float(x[2]).hex()]
    t = x[0][:4+to] + x[0][-4:]
    l = x[1][:4+to] + x[1][-4:]
    n = x[2][:4+to] + x[2][-4:]
    return [t, l, n]
def r_hex(x, to=5):
    x = float(x).hex()
    return x[:4+to] + x[-4:]


def print_hex(x):
  x_print = [float.hex(x[0]), float.hex(x[1]), float.hex(x[2])]
  return x_print


phi = (1 + m.sqrt(5))/2

def gold_ratio(i_num, x, complexity):

  b_ = b
  a_ = a
  x1 = b_ - (b_ - a_)/phi
  x2 = a_ + (b_ - a_)/phi
  num_iter = 1
  complexity = 0

  f_1, complexity = f(x1, i_num, x, complexity)
  f_2, complexity = f(x2,i_num, x,complexity)
  true_right = 1
  while ((b_ - a_)/2 >= eps):
    num_iter += 1
    #complexity = 0
    if (f_1 >= f_2):
      a_ = x1
      x1 = x2

      x2 = b_ - (x1 - a_)
      f_1 = f_2
      f_2, complexity = f(x2,i_num, x,complexity)
      true_right = 0
    else:
      b_ = x2
      x2 = x1
      x1 = a_ + (b_ - x2)
      f_2 = f_1
      f_1, complexity = f(x1,i_num, x,complexity)
      true_right = 1

  acc = (b_ - a_)/2
  answ, complexity = f((a_ + b_)/2, i_num, x, complexity)
#  if (true_right):
#    print("истина справа")
#  else:
#    print("истина слева")
  return (a_ + b_)/2, complexity

def coord_descent(n):
  complexity = 0
  x = np.zeros(n)
  x = [-1, -4, -4]
  new_x = x
  delta = 1
  while (delta >= eps):
    x = new_x
    for i in range (n):
      new_x[i], complexity = gold_ratio(i, x, complexity)
    delta = linalg.norm(new_x - x)
  print (new_x)
  answ, complexity = f(0, -1, new_x, complexity)
  print("complexity = ", complexity)
  return answ
def gradient_descent_not(n):
  np.set_printoptions(formatter={'int':hex})
  complexity = 0
  complexity_df = 0
  iter = 0
  x = np.zeros(n)

  df = np.zeros(n)

  new_x = x
  delta = 1
  alpha = 1

  df_k = df
  iter = 0
  print("iter\t\t\t\t","grad\t\t\t\t", "grad_norm\t\t", "funct\t\t", "c_f\t", "c_df")
  while (delta >= eps):
    #np.set_printoptions(formatter={'int':hex})
    df, complexity_df = df_(df, x, complexity_df)
    x_ = x - alpha * df
    f_xk, complexity = f(0, -1, x, complexity)
    df_k, complexity_df = df_(df_k, x, complexity_df)

    f_x, complexity = f(0, -1, x_, complexity)
    if ((f_x - f_xk) > (-alpha/(16**4) * linalg.norm(df_k)**2)):
      alpha = alpha / 16
    else:
      new_x = x - alpha * df_k
      delta = linalg.norm(new_x - x)
      x = new_x

    #new_x = x - alpha * df_k
    #delta = linalg.norm(new_x - x)
    #x = new_x

    iter += 1
    #np.set_printoptions(formatter={'all': lambda x: f'{hex(x)}'})
    print(iter,"\t", round_hex(df_k), "\t", linalg.norm(df_k).hex(),"\t", f_xk,"\t", alpha, "\t", complexity, "\t", complexity_df)


  print (x)
  answ, complexity = f(0, -1, x, complexity)
  return (answ)
def gradient_descent_const(n):
  complexity = 0
  complexity_df = 0
  x = np.zeros(n)
  df = np.zeros(n)

  new_x = x
  delta = 1
  alpha = 0.1
  iter = 0
  print("iter\t\t\t\t","grad\t\t\t\t", "grad_norm\t\t", "funct\t\t\t", "c_f\t", "c_df")

  while (delta >= eps):
    x = new_x
    df, complexity_df = df_(df, x, complexity_df)
    new_x = x - alpha * df
    f_x, complexity = f(0, -1, x, complexity)

    delta = linalg.norm(new_x - x)
    #prom, complexity = f(0, -1, x, complexity)
    #promnew, complexity = f(0, -1, new_x, complexity)
    #delta = linalg.norm(promnew - prom)
    iter += 1
    #np.set_printoptions(formatter={'all': lambda x: f'{hex(x)}'})
    print(iter,"\t", round_hex(df), "\t", linalg.norm(df).hex(),"\t", f_x,"\t", complexity, "\t", complexity_df)

  print (new_x)
  answ, complexity = f(0, -1, new_x, complexity)
  return (answ)


def gradient_descent_not(n, alpha):
  np.set_printoptions(formatter={'int':hex})
  complexity = 0
  complexity_df = 0
  iter = 0
  x = np.zeros(n)
  x = [-1, -5, -4]
  df = np.zeros(n)
  grad2 = np.zeros(n)

  new_x = x
  delta = 1
  #alpha = 2
  grad2 = df_2(grad2, [0,0,0])
  min_gr = min(grad2)
  df_k = df
  iter = 0
  delta2 = 0
  print("iter\t\t\t\t", "m\t\t\t\t","grad\t\t\t", "grad_norm\t\t", "funct\t\t\t", "c_f\t", "c_df")
  while (delta >= eps):
    #np.set_printoptions(formatter={'int':hex})
    df, complexity_df = df_(df, x, complexity_df)

    x_ = x - alpha * df
    f_xk, complexity = f(0, -1, x, complexity)
    df_k, complexity_df = df_(df_k, x, complexity_df)

    f_x, complexity = f(0, -1, x_, complexity)
    if ((f_x - f_xk) > (-alpha/(16**4) * linalg.norm(df_k)**2)):
      alpha = alpha / 2
    else:
      new_x = x - alpha * df_k
      delta = linalg.norm(new_x - x)

      #delta = linalg.norm(df)
      x = new_x
    #delta = linalg.norm(new_x - x)

    #delta = linalg.norm(df)/min(grad2)
    #new_x = x - alpha * df_k
    #delta = linalg.norm(new_x - x)
    #x = new_x

    iter += 1

    #np.set_printoptions(formatter={'all': lambda x: f'{hex(x)}'})
    print(iter,"  ", round_hex(x),"\t", round_hex(df_k), "\t", r_hex(linalg.norm(df_k)),"\t", f_xk,"\t", complexity, " ", complexity_df)

  print(round_hex(x))
  answ, complexity = f(0, -1, x, complexity)
  return (answ)


def gradient_descent_const(n, alpha):
  complexity = 0
  complexity_df = 0
  x = np.zeros(n)
  x = [-1, -5, -4]
  df = np.zeros(n)
  f_x = np.zeros(n)
  grad2 = np.zeros(n)

  new_x = x
  delta = 1
  #alpha = 2.45
  iter = 0
  print("iter\t\t\t\t","m\t\t\t\t" ,"grad\t\t\t\t", "grad_norm\t\t", "funct\t\t", "c_f\t", "c_df")

  grad2 = df_2(grad2, [0,0,0])
  min_gr = abs(min(grad2))

  while (delta >= eps):
    x = new_x
    delta2 = -df
    df, complexity_df = df_(df, x, complexity_df)
    new_x = x - alpha * df
    f_x, complexity = f(0, -1, x, complexity)

    #grad_2 = df_2(grad_2, x)

    delta = linalg.norm(df)/min_gr


    #delta = linalg.norm(promnew - prom)
    #delta2 += df

    #delta = linalg.norm(delta2)
    iter += 1
    #np.set_printoptions(formatter={'all': lambda x: f'{hex(x)}'})
    print(iter,"  ", round_hex(x), "\t", round_hex(df), "\t", r_hex(linalg.norm(df)),"\t", f_x,"\t", complexity, " ", complexity_df)


  print(round_hex(x))
  answ, complexity = f(0, -1, x, complexity)
  return (answ)


In [ ]:
gradient_descent_not(n, 2.4)

In [ ]:
print(sp.optimize.minimize(f_1, np.array([-1, -4, -4]), method = 'SLSQP'))

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 5.264568718940991
       x: [-8.023e-01 -4.810e+00 -4.371e+00]
     nit: 15
     jac: [-2.451e-04  5.293e-05  9.358e-05]
    nfev: 60
    njev: 15


In [1]:
import numpy as np
import pandas as pd
import sympy as sym
import scipy.linalg as sla
import matplotlib.pyplot as plt
from numpy import linalg
from sklearn import datasets
from sklearn.linear_model import LinearRegression, Lasso, Ridge


eps = 1/(16**4/2)
n = 3

h = 1/(16**3*2)

k_iter = 0
complexity = 0

def round_hex(x, to=5):
    x = [float(x[0]).hex(), float(x[1]).hex()]
    t = x[0][:4+to] + x[0][-4:]
    l = x[1][:4+to] + x[1][-4:]
    return [t, l, n]
def r_hex(x, to=5):
    x = float(x).hex()
    return x[:4+to] + x[-4:]


def print_hex(x):
  x_print = [float.hex(x[0]), float.hex(x[1]), float.hex(x[2])]
  return x_print



eps = 0.00001
eps1 = 0.001
n = 2
deriv = np.zeros(n)
def f(x, complexity):
    complexity += 1
    return np.cosh(0.6*x[0]) + np.cosh(0.1*x[1]) + x[0] + 11*x[1]
def d_f(x, complexity):
    complexity += 1
    deriv[0] = 0.6*np.sinh(0.6*x[0]) + 1
    deriv[1] = 0.1*np.sinh(0.1*x[1]) + 11
    return deriv
def g(x):
    return x[0] - (0.5)*x[1]
def g2(x):
    return x[0]**2 + x[1]**2 -3

def H(x):
    return (max(0, g(x)))**2 + (max(0, g2(x)))**2
def d_h(x):
    deriv[0] = 2*(max(0, g(x)))*(1) + 2*(max(0, g2(x)))*2*x[0]
    deriv[1] = 2*(max(0, g(x)))*(-0.5) + 2*(max(0, g2(x)))*2*x[0]
    return deriv
def phi(x, r, complexity):
    complexity += 1
    return (f(x, complexity) + r * H(x))
def d_phi(x, r, complexity):
    deriv = d_f(x, complexity) + r*d_h(x)
    return deriv

def gradient_descent_const(n, r, x0):
  complexity = 0
  complexity_df = 0
  x = x0
  df = np.zeros(n)

  new_x = x
  delta = 1
  alpha = 1
  iter = 0
  eps = 0.01

  while (delta >= eps):
    x = new_x
    #df[0] = (phi([x[0] + 0.000001, x[1]], r, complexity) - phi(x, r, complexity)) / (0.000001)
    #df[1] = (phi([x[0], x[1] + 0.000001], r, complexity) - phi(x, r, complexity)) / (0.000001)
    df = d_phi(x, r, complexity)
    new_x = x - alpha * df

    delta = linalg.norm(new_x - x)
    iter += 1

  return new_x

def gradient_descent_const_JohnIsAGenius(x, r, phi):
  complexity = 0
  complexity_df = 0
  df = np.zeros(2)

  new_x = x
  delta = 1
  alpha = 0.1
  iter = 0
  eps = 0.001

  while (delta >= eps):
    x = new_x
    df[0] = (phi([x[0] + 0.000001, x[1]], r,complexity) - phi(x, r, complexity)) / (0.000001)
    df[1] = (phi([x[0], x[1] + 0.000001], r, complexity) - phi(x, r, complexity)) / (0.000001)
    new_x = x - alpha * df

    delta = linalg.norm(new_x - x)
    iter += 1

  return new_x

def gradient_descent_not(n, r, x0):
  complexity = 0
  complexity_df = 0

  x = x0
  df = np.zeros(n)

  new_x = x
  delta = 1
  alpha = 0.94
  eps = 0.001
  df_k = df
  iter = 0
  print("iter\t\t\t\t", "m\t\t\t\t","grad\t\t\t", "grad_norm\t\t", "funct\t", "g1\t\t", "g2\t")
  #"c_f\t", "c_df"
  while (delta >= eps):

    df = d_phi(x, r, complexity)
    x_ = x - alpha * df
    f_xk = phi(x, r, complexity)
    df_k = d_phi(x, r, complexity_df)

    f_x = phi(x_, r, complexity)
    if ((f_x - f_xk) > (-alpha/(16**4) * linalg.norm(df_k)**2)):
      alpha = alpha / 19
    else:
      new_x = x - alpha * df_k
      delta = linalg.norm(new_x - x)
      x = new_x

    iter += 1
    #np.set_printoptions(formatter={'all': lambda x: f'{hex(x)}'})
    print(iter,"  ", round_hex(x), "\t", round_hex(df), "\t", r_hex(linalg.norm(df)),"\t", r_hex(f_x),"\t", r_hex(g(x)), "\t", r_hex(g2(x)))

  return (x)


def external_penalty(x0):
    x_ = x0
    r = 1
    delta = 1
    ITER = 0
    while (delta >= eps):
      print("ITER = ", ITER)
      ITER += 1
      print("x_ = ", x_)

      x_ = gradient_descent_not(n, r, x_)

      print("\nf\t\t\t", "g1\t\t\t\t", "g2\t\t\t\t")
      print(r_hex(f(x_, complexity))),"  ", (r_hex(g(x_))), "\t", (r_hex(g2(x_)))
      print("r = " , r)
      print("x_ = ", round_hex(x_))
      r *= 10
      print("\n\n\n")
      delta = H(x_)
      #print(delta)
      #print(x_)
    print("\nf\t\t\t", "g1\t\t\t\t", "g2\t\t\t\t")
    #print((float(f(x_, complexity)).hex()),"  ", (float(g(x_)).hex()), "\t", (float(g2(x_)).hex()))
    print(r_hex(f(x_, complexity))),"  ", (r_hex(g(x_))), "\t", (r_hex(g2(x_)))
    #print((f(x_, complexity)),"  ", (g(x_)), "\t", (g2(x_)))
    print("delta = ", delta)
    return round_hex(x_[:2])

x0 = [-0.7, -2]
external_penalty(x0)




ITER =  0
x_ =  [-0.7, -2]
iter				 m				 grad			 grad_norm		 funct	 g1		 g2	
1    ['-0x1.66666p-1', '-0x1.00000p+1', 2] 	 ['-0x1.c937bp+2', '-0x1.1e357p+3', 2] 	 0x1.6e4d2cp+3 	 0x1.5ef7fp+12 	 0x1.333334p-2 	 0x1.7d70a8p+0
2    ['-0x1.66666p-1', '-0x1.00000p+1', 2] 	 ['-0x1.c937bp+2', '-0x1.1e357p+3', 2] 	 0x1.6e4d2cp+3 	 -0x1.e8454p+3 	 0x1.333334p-2 	 0x1.7d70a8p+0
3    ['-0x1.5ce05p-1', '-0x1.fa092p+0', 2] 	 ['-0x1.c937bp+2', '-0x1.1e357p+3', 2] 	 0x1.6e4d2cp+3 	 -0x1.25853p+4 	 0x1.3a531ap-2 	 0x1.5f2700p+0
4    ['-0x1.548bdp-1', '-0x1.f4a4bp+0', 2] 	 ['-0x1.8ff70p+2', '-0x1.02eb9p+3', 2] 	 0x1.4728bfp+3 	 -0x1.25fd4p+4 	 0x1.40336cp-2 	 0x1.445560p+0
5    ['-0x1.4d393p-1', '-0x1.efbbdp+0', 2] 	 ['-0x1.5f647p+2', '-0x1.d778ap+2', 2] 	 0x1.2601fcp+3 	 -0x1.260cfp+4 	 0x1.4504f4p-2 	 0x1.2c6794p+0
6    ['-0x1.4d393p-1', '-0x1.efbbdp+0', 2] 	 ['-0x1.35c4ap+2', '-0x1.afa6ap+2', 2] 	 0x1.09a65ep+3 	 -0x1.25cf1p+4 	 0x1.4504f4p-2 	 0x1.2c6794p+0
7    ['-0x1.4d393p-1', '-0x1.efbbdp+0', 

['-0x1.813e1p-1', '-0x1.820a4p+0', 2]